In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
from src.utils.outliers.iqr import add_iqr_outs_to_df, plot_iqr_outs, plot_iqr_outs_cls
from src.utils.outliers.pyod import add_pyod_outs_to_df, plot_pyod_outs, plot_pyod_outs_regression_error
from scripts.python.dataset_specific.GSEUNN.tasks.routines_046 import plot_regression_error_distributions, plot_cls_dim_red
from plotly.subplots import make_subplots
from scipy import stats
import plotly.express as px
from scripts.python.routines.plot.scatter import add_scatter_trace
import plotly.graph_objects as go
from scripts.python.routines.plot.save import save_figure
from scripts.python.routines.plot.layout import add_layout, get_axis
from statsmodels.stats.multitest import multipletests
import plotly.io as pio
import importlib
pio.kaleido.scope.mathjax = None
from plotly.offline import init_notebook_mode, iplot
from scipy.interpolate import interp1d
from src.utils.verbose import NoStdStreams
init_notebook_mode(connected=False)
import matplotlib.pyplot as plt
from matplotlib import colors
from omegaconf import OmegaConf
from tqdm import tqdm
import seaborn as sns
from glob import glob
import pathlib
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.decomposition import MiniBatchDictionaryLearning, FastICA
from sklearn.random_projection import GaussianRandomProjection, SparseRandomProjection
from sklearn.manifold import MDS, Isomap
from openTSNE import TSNE
from sklearn.metrics import mean_absolute_error
from scipy import stats
import patchworklib as pw
import os
import functools
from scipy.stats import iqr
from statannotations.Annotator import Annotator
from scipy.stats import mannwhitneyu
import shap
from slugify import slugify
from src.models.tabular.widedeep.ft_transformer import WDFTTransformerModel
from src.models.tabular.widedeep.tab_net import WDTabNetModel
from art.estimators.regression.pytorch import PyTorchRegressor
from art.estimators.classification import PyTorchClassifier
from art.estimators.regression.blackbox import BlackBoxRegressor
from art.attacks.evasion import ProjectedGradientDescentNumpy, FastGradientMethod, BasicIterativeMethod, MomentumIterativeMethod
from art.attacks.evasion import ZooAttack, CarliniL2Method, ElasticNet, NewtonFool
import torch
from src.tasks.metrics import get_cls_pred_metrics, get_cls_prob_metrics
import matplotlib.lines as mlines

from sdv.metadata import SingleTableMetadata
from sdv.lite import SingleTablePreset
from sdv.single_table import GaussianCopulaSynthesizer, CTGANSynthesizer, TVAESynthesizer, CopulaGANSynthesizer
from sdv.evaluation.single_table import evaluate_quality
from sdv.evaluation.single_table import get_column_plot
from sdv.evaluation.single_table import get_column_pair_plot
from sklearn.preprocessing import StandardScaler
from scripts.python.routines.mvals import expit2

import missingno as msno

import joblib
import pickle

from pyod.models.ecod import ECOD
from pyod.models.abod import ABOD
from pyod.models.copod import COPOD
from pyod.models.sos import SOS
from pyod.models.kde import KDE
from pyod.models.sampling import Sampling
from pyod.models.gmm import GMM

from pyod.models.kpca import KPCA
from pyod.models.mcd import MCD
from pyod.models.ocsvm import OCSVM
from pyod.models.lmdd import LMDD

from pyod.models.lof import LOF
from pyod.models.cof import COF
from pyod.models.cblof import CBLOF
from pyod.models.hbos import HBOS
from pyod.models.knn import KNN
from pyod.models.sod import SOD

from pyod.models.iforest import IForest
from pyod.models.inne import INNE
from pyod.models.loda import LODA
from pyod.models.suod import SUOD

from pyod.models.auto_encoder_torch import AutoEncoder
from pyod.models.vae import VAE
from pyod.models.deep_svdd import DeepSVDD

from pyod.models.lunar import LUNAR

from torchmetrics import BootStrapper


def conjunction(conditions):
    return functools.reduce(np.logical_and, conditions)


def disjunction(conditions):
    return functools.reduce(np.logical_or, conditions)

# 2. Adversarial examples for DNAm data

## Data processing

### Prepare data for ML, convert mvals to betas

In [ ]:
path = "D:/YandexDisk/Work/pydnameth/datasets/GPL21145/GSEUNN/special/046_adversarial_robustness_toolbox/dnam"

df_mvals = pd.read_excel(f"{path}/mvals.xlsx", index_col=0)
feats = pd.read_excel(f"{path}/feats_1000.xlsx", index_col=0).index.values

betas = expit2(df_mvals.loc[:, feats].values)
df_betas = df_mvals.copy()
df_betas.loc[:, feats] = betas
df_betas['Partition'].replace({'Train': 'trn_val', 'Validation': 'tst'}, inplace=True)
df_betas.to_excel(f"{path}/betas.xlsx", index_label='subject_id')

### Collect ML results

In [ ]:
path = "D:/YandexDisk/Work/pydnameth/datasets/GPL21145/GSEUNN/special/046_adversarial_robustness_toolbox/dnam"

model = 'widedeep_tab_net'

path_runs = f"{path}/models/{model}_trn_val_tst/multiruns"

files = glob(f"{path_runs}/*/*/metrics_all_best_*.xlsx")

df_tmp = pd.read_excel(files[0], index_col="metric")
head, tail = os.path.split(files[0])
cfg = OmegaConf.load(f"{head}/.hydra/overrides.yaml")
params = []
for param_pair in cfg:
    param, val = param_pair.split('=')
    params.append(param)
df_res = pd.DataFrame(index=files)

parts = [
    'trn',
    'val',
    'tst',
    'val_tst'
]

for file in files:
    head, tail = os.path.split(file)
    # Metrics
    df_metrics = pd.read_excel(file, index_col="metric")
    for metric in df_metrics.index.values:
        for part in parts:
            df_res.at[file, metric + f"_{part}"] = df_metrics.at[metric, part]

    # Params
    cfg = OmegaConf.load(f"{head}/.hydra/overrides.yaml")
    for param_pair in cfg:
        param, val = param_pair.split('=')
        df_res.at[file, param] = val

df_res["train_worse_val"] = False
df_res.loc[df_res["accuracy_weighted_val"] > df_res["accuracy_weighted_trn"], "train_worse_val"] = True

df_res["File"] = df_res.index.str.replace(path_runs, '', regex=False)
df_res.set_index("File", inplace=True)

first_columns = [
    'accuracy_weighted_trn',
    'accuracy_weighted_val',
    'accuracy_weighted_tst',
    'accuracy_weighted_val_tst'
]
df_res = df_res[first_columns + [col for col in df_res.columns if col not in first_columns]]
df_res.to_excel(f"{path_runs}/summary.xlsx", index=True, index_label="file")

### Load data, models

In [ ]:
model_type = 'widedeep_tab_net'
model_fn = 'best_fold_0000'
model_version = 'v2'

path = 'D:/YandexDisk/Work/pydnameth/datasets/GPL21145/GSEUNN/special/046_adversarial_robustness_toolbox/dnam'
pathlib.Path(f"{path}/Origin").mkdir(parents=True, exist_ok=True)
df = pd.read_excel(f"{path}/betas.xlsx", index_col=0)
feats = pd.read_excel(f"{path}/feats_1000.xlsx", index_col=0).index.values
ids_feat = list(range(len(feats)))

df_pred = pd.read_excel(f"{path}/models/{model_type}/{model_version}/predictions.xlsx", index_col=0)
df.loc[df.index, ['Real', 'Pred', 'Prob Control', 'Prob Parkinson']] = df_pred.loc[df.index, ['Status', 'pred', 'pred_prob_0', 'pred_prob_1']].values
df['Data'] = 'Real'
df['Eps'] = 'Origin'

col_real = 'Real'
col_pred = 'Pred'

ids_trn_val = df.index[df['Partition'] == 'trn_val'].values
ids_tst = df.index[df['Partition'] == 'tst'].values
ids_all = df.index[df['Partition'].isin(['trn_val', 'tst'])].values
ids_dict = {
    'trn_val': ids_trn_val,
    'tst': ids_tst,
    'all': ids_all,
}

model = WDTabNetModel.load_from_checkpoint(checkpoint_path=f"{path}/models/{model_type}/{model_version}/{model_fn}.ckpt")
model.produce_probabilities = False
model.eval()
model.freeze()

colors_augs = {
    'FAST_ML': px.colors.qualitative.Light24[0],
    'GaussianCopula': px.colors.qualitative.Light24[1],
    'CTGANSynthesizer': px.colors.qualitative.Light24[2],
    'TVAESynthesizer': px.colors.qualitative.Light24[3],
    'CopulaGANSynthesizer': px.colors.qualitative.Light24[4],
}
colors_atks_eps = {
    "MomentumIterative": px.colors.qualitative.D3[0],
    "BasicIterative": px.colors.qualitative.D3[1],
    "ProjectedGradientDescent": px.colors.qualitative.D3[2],
    "FastGradient": px.colors.qualitative.D3[3],
}
colors_atks_bss = {
    "ElasticNet": px.colors.qualitative.G10[7],
    "CarliniL2Method": px.colors.qualitative.G10[8],
    "ZooAttack": px.colors.qualitative.G10[9],
}
colors_atks_eta = {
    'NewtonFool': px.colors.qualitative.T10[7],
}

dim_red_labels = {
    'PCA': ['PC 1', 'PC 2'],
    'SVD': ['SVD 1', 'SVD 2'],
    't-SNE': ['t-SNE 1', 't-SNE 2'],
    'GRP': ['GRP 1', 'GRP 2'],
    'SRP': ['SRP 1', 'SRP 2'],
    'IsoMap': ['IsoMap 1', 'IsoMap 2'],
    'MBDL': ['MBDL 1', 'MBDL 2'],
}

pyod_method_names = [
    'ECOD',
    'LUNAR',
    'DeepSVDD',
    'VAE',
    'LODA',
    'INNE',
    'IForest',
    'SOD',
    'KNN',
    'CBLOF',
    'LOF',
    'MCD',
    'GMM',
    'Sampling',
    'SOS',
    'COPOD',
]

thld_iqr_in = 10
thld_iqr_out = 100
thld_outs_pyod = 1/3

### Create PyOD models, trained on trn_val samples

In [ ]:
contamination = 0.1

pyod_methods = {
    'ECOD': ECOD(contamination=contamination),
    'LUNAR': LUNAR(),
    'DeepSVDD': DeepSVDD(contamination=contamination, verbose=0),
    'VAE': VAE(encoder_neurons=[32, 16, 8], decoder_neurons=[8, 16, 32], contamination=contamination),
    'LODA': LODA(contamination=contamination),
    'INNE': INNE(contamination=contamination),
    'IForest': IForest(contamination=contamination),
    'SOD': SOD(contamination=contamination),
    'KNN': KNN(contamination=contamination),
    'CBLOF': CBLOF(contamination=contamination),
    'LOF': LOF(contamination=contamination),
    'MCD': MCD(contamination=contamination),
    'GMM': GMM(contamination=contamination),
    'Sampling': Sampling(contamination=contamination),
    'SOS': SOS(contamination=contamination),
    'COPOD': COPOD(contamination=contamination),
}

for method_name, method in (pbar := tqdm(pyod_methods.items())):
    pbar.set_description(f"Processing {method_name}")
    
    method.fit(df.loc[ids_trn_val, feats].values)

### Create dimensionality reduction models, trained on trn_val samples

In [ ]:
X_dim_red = df.loc[ids_trn_val, feats].values
random_state = 42
dim_red_models = {
    'PCA': PCA(n_components=2, whiten=False, random_state=random_state).fit(X_dim_red),
    'SVD': TruncatedSVD(n_components=2, algorithm='randomized', n_iter=5, random_state=random_state).fit(X_dim_red),
    't-SNE': TSNE(n_components=2, random_state=random_state).fit(X_dim_red),
    'GRP': GaussianRandomProjection(n_components=2, eps=0.5, random_state=random_state).fit(X_dim_red),
    'SRP': SparseRandomProjection(n_components=2, density='auto', eps=0.5, dense_output=False, random_state=random_state).fit(X_dim_red),
    'IsoMap': Isomap(n_components=2, n_neighbors=5).fit(X_dim_red),
    'MBDL': MiniBatchDictionaryLearning(n_components=2, batch_size=100, alpha=1, n_iter=25, random_state=random_state).fit(X_dim_red),
}

### Original data processing

In [ ]:
add_pyod_outs_to_df(df, pyod_methods, feats)
add_iqr_outs_to_df(df, df.loc[ids_trn_val, :], feats)
for method_name, method in (pbar := tqdm(dim_red_models.items())):
    pbar.set_description(f"Processing {method_name}")
    dim_red_res = method.transform(df.loc[:, feats].values)
    df.loc[:, dim_red_labels[method_name][0]] = dim_red_res[:, 0]
    df.loc[:, dim_red_labels[method_name][1]] = dim_red_res[:, 1]
df.to_excel(f"{path}/Origin/df.xlsx", index_label='sample_id')

### Load original processed data

In [ ]:
df = pd.read_excel(f"{path}/Origin/df.xlsx", index_col=0)

### Original data plots in reduced dimension

In [ ]:
pathlib.Path(f"{path}/Origin/dim_red").mkdir(parents=True, exist_ok=True)

df_fig = df.loc[:, list(np.concatenate(list(dim_red_labels.values()))) + ['Real', 'Pred', 'Prob Parkinson']].copy()
df_fig.loc[df_fig['Real'] == 0, 'Status'] = 'Control'
df_fig.loc[df_fig['Real'] == 1, 'Status'] = 'Parkinson'
for method in dim_red_labels:
    plot_cls_dim_red(
        df=df_fig,
        col_class='Status',
        cls_names=['Control', 'Parkinson'],
        col_prob='Prob Parkinson',
        cols_dim_red=dim_red_labels[method],
        title='Original',
        fn=f"{path}/Origin/dim_red/{method}"
    )

In [ ]:
pathlib.Path(f"{path}/Origin/feats").mkdir(parents=True, exist_ok=True)

df_stat = pd.DataFrame(index=feats, columns=['mw_pval', 'mw_pval_fdr_bh'])
for f in feats:
    _, df_stat.at[f, 'mw_pval'] = mannwhitneyu(df.loc[df['Real'] == 0, f].values, df.loc[df['Real'] == 1, f].values, alternative='two-sided')
_, df_stat.loc[:, 'mw_pval_fdr_bh'], _, _ = multipletests(df_stat.loc[:, "mw_pval"], 0.05, method='fdr_bh')
df_stat.sort_values(['mw_pval_fdr_bh'], ascending=[True], inplace=True)
df_stat[r'$ -\log_{10}(\mathrm{p-value})$'] = -np.log10(df_stat['mw_pval_fdr_bh'].astype(float))
df_stat.to_excel(f"{path}/Origin/feats/df_stat.xlsx", index_label="Features")

fig, ax = plt.subplots(figsize=(10, 4))
sns.set_theme(style='whitegrid')
kdeplot = sns.kdeplot(
    data=df_stat,
    x=r'$ -\log_{10}(\mathrm{p-value})$',
    color='darkgreen',
    linewidth=2,
    cut=0,
    fill=True,
    ax=ax
)
kdeplot.set_title('Features Distribution Differences')
plt.savefig(f"{path}/Origin/feats/kde_pval.png", bbox_inches='tight', dpi=200)
plt.savefig(f"{path}/Origin/feats/kde_pval.pdf", bbox_inches='tight')
plt.close(fig)

In [ ]:
n_top_features = 10
top_features = list(df_stat.index[0:n_top_features])
df_fig = df.loc[:, top_features + ['Real']].copy()
df_fig.loc[df_fig['Real'] == 0, 'Status'] = 'Control'
df_fig.loc[df_fig['Real'] == 1, 'Status'] = 'Parkinson'
df_fig = df_fig.melt(id_vars=['Status'], value_vars=list(df_stat.index[0:n_top_features]), var_name='CpG', value_name='Methylation')
df_fig['CpG'].replace({x: f"{x}\npval: {df_stat.at[x, 'mw_pval_fdr_bh']:0.2e}" for x in top_features}, inplace=True)

fig, ax = plt.subplots(figsize=(10, 1 * n_top_features))
sns.set_theme(style='whitegrid')
violin = sns.violinplot(
    data=df_fig,
    x='Methylation',
    y='CpG',
    orient='h',
    hue='Status',
    split=True,
    linewidth=1,
    palette={'Control': 'dodgerblue', 'Parkinson': 'crimson'},
    hue_order=['Control', 'Parkinson'],
    cut=0,
    inner="quart",
    ax=ax
)
plt.savefig(f"{path}/Origin/feats/violins.png", bbox_inches='tight', dpi=200)
plt.savefig(f"{path}/Origin/feats/violins.pdf", bbox_inches='tight')
plt.close(fig)

In [ ]:
df_fig = df.loc[:, top_features + ['Prob Parkinson', 'Real']].copy()
df_fig.loc[df_fig['Real'] == 0, 'Status'] = 'Control'
df_fig.loc[df_fig['Real'] == 1, 'Status'] = 'Parkinson'
fig, ax = plt.subplots(figsize=(10, 4))
sns.set_theme(style='whitegrid')
kdeplot = sns.kdeplot(
    data=df_fig,
    x='Prob Parkinson',
    hue='Status',
    palette={'Control': 'dodgerblue', 'Parkinson': 'crimson'},
    hue_order=['Control', 'Parkinson'],
    linewidth=2,
    cut=0,
    fill=True,
    ax=ax
)
plt.savefig(f"{path}/Origin/feats/kde_proba.png", bbox_inches='tight', dpi=200)
plt.savefig(f"{path}/Origin/feats/kde_proba.pdf", bbox_inches='tight')
plt.close(fig)

### Outliers analysis

In [ ]:
# IQR plots
pathlib.Path(f"{path}/Origin/outliers_iqr").mkdir(parents=True, exist_ok=True)
plot_iqr_outs(df, feats, 'grey', 'Origin', f"{path}/Origin/outliers_iqr", is_msno_plots=False)
df_fig = df.loc[:, ['Real', 'Pred', 'n_outs_iqr', 'Prob Control', 'Prob Parkinson']].copy()
df_fig.loc[df_fig['Real'] == 0, 'Status'] = 'Control'
df_fig.loc[df_fig['Real'] == 1, 'Status'] = 'Parkinson'
plot_iqr_outs_cls(
    df=df_fig,
    path=f"{path}/Origin/outliers_iqr",
    thld_in=thld_iqr_in,
    thld_out=thld_iqr_out,
    col_class="Status",
    col_pred="Pred",
    col_real="Real",
    cols_prob=['Prob Control', 'Prob Parkinson'],
    palette={'Control': 'dodgerblue', 'Parkinson': 'crimson'}
)

# PyOD plots
pathlib.Path(f"{path}/Origin/outliers_pyod").mkdir(parents=True, exist_ok=True)
plot_pyod_outs(df, pyod_methods, 'grey', 'Origin', f"{path}/Origin/outliers_pyod", n_cols=4)

## 2.2 Augmented data

In [ ]:
n_smps = 1000

df_aug_sdv_input = df.loc[:, np.concatenate((['Real'], feats))]
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(data=df_aug_sdv_input)
metadata.update_column('Real', sdtype='categorical')

synthesizers = {
    'GaussianCopula': GaussianCopulaSynthesizer(metadata),
    'CTGANSynthesizer': CTGANSynthesizer(metadata),
    'TVAESynthesizer': TVAESynthesizer(metadata),
    'CopulaGANSynthesizer': CopulaGANSynthesizer(metadata),
    'FAST_ML': SingleTablePreset(metadata, name='FAST_ML'),
}

for s_name, s in (pbar := tqdm(synthesizers.items())):
    pbar.set_description(f"Processing {s_name}")
    path_curr = f"{path}/Augmentation/{s_name}"
    pathlib.Path(f"{path_curr}").mkdir(parents=True, exist_ok=True)

    s.fit(
        data=df_aug_sdv_input
    )
    s.save(
        filepath=f"{path_curr}/synthesizer.pkl"
    )
    df_aug_sdv = s.sample(
        num_rows=n_smps
    )
    quality_report = evaluate_quality(
        df_aug_sdv_input,
        df_aug_sdv,
        metadata
    )
    
    q_rep_prop = quality_report.get_properties()
    q_rep_prop.set_index('Property', inplace=True)
    
    df_col_shapes = quality_report.get_details(property_name='Column Shapes')
    df_col_shapes.sort_values(["Score"], ascending=[False], inplace=True)
    df_col_shapes.to_excel(f"{path_curr}/ColumnShapes.xlsx", index=False)
    
    df_col_pair_trends = quality_report.get_details(property_name='Column Pair Trends')
    df_col_pair_trends.to_excel(f"{path_curr}/ColumnPairTrends.xlsx", index=False)
    
    model.produce_probabilities = True
    y_pred_prob = model(torch.from_numpy(np.float32(df_aug_sdv.loc[:, feats].values))).cpu().detach().numpy()
    y_pred = np.argmax(y_pred_prob, 1)
    df_aug_sdv["Pred"] = y_pred
    df_aug_sdv["Prob Control"] = y_pred_prob[:, 0]
    df_aug_sdv["Prob Parkinson"] = y_pred_prob[:, 1]
    
    for m, drm in dim_red_models.items():
        dim_red_res = drm.transform(df_aug_sdv.loc[:, feats].values)
        df_aug_sdv.loc[:, dim_red_labels[m][0]] = dim_red_res[:, 0]
        df_aug_sdv.loc[:, dim_red_labels[m][1]] = dim_red_res[:, 1]
        
    add_iqr_outs_to_df(df_aug_sdv, df.loc[ids_trn_val, :], feats)
    add_pyod_outs_to_df(df_aug_sdv, pyod_methods, feats)
        
    df_aug_sdv.to_excel(f"{path_curr}/df.xlsx", index_label='sample_id')


## 2.3. Attacks

In [ ]:
ids_atk = ids_tst

art_classifier = PyTorchClassifier(
    model=model,
    loss=model.loss_fn,
    input_shape=(len(feats),),
    nb_classes=2,
    optimizer=torch.optim.Adam(
        params=model.parameters(),
        lr=model.hparams.optimizer_lr,
        weight_decay=model.hparams.optimizer_weight_decay
    ),
    use_amp=False,
    opt_level="O1",
    loss_scale="dynamic",
    channels_first=True,
    clip_values=(0.0, 1.0),
    preprocessing_defences=None,
    postprocessing_defences=None,
    preprocessing=(0.0, 1.0),
    device_type="cpu"
)

### Eps-depended attacks

In [ ]:
epsilons = sorted(list(set.union(
    set(np.linspace(0.1, 1.0, 10)), 
    set(np.linspace(0.01, 0.1, 10)),
    set(np.linspace(0.001, 0.01, 10))
)))
df_eps = pd.DataFrame(index=epsilons)

for eps_raw in epsilons:

    eps = np.array([eps_raw * iqr(df.loc[ids_atk, feat].values) for feat in feats])
    eps_step = np.array([0.2 * eps_raw * iqr(df.loc[ids_atk, feat].values) for feat in feats])

    attacks = {
        'MomentumIterative': MomentumIterativeMethod(
            estimator=art_classifier,
            norm=np.inf,
            eps=eps,
            eps_step=eps_step,
            decay=0.1,
            max_iter=100,
            targeted=False,
            batch_size=512,
            verbose=True
        ),
        'BasicIterative': BasicIterativeMethod(
            estimator=art_classifier,
            eps=eps,
            eps_step=eps_step,
            max_iter=100,
            targeted=False,
            batch_size=512,
            verbose=True
        ),
        'ProjectedGradientDescent': ProjectedGradientDescentNumpy(
            estimator=art_classifier,
            norm=np.inf,
            eps=eps,
            eps_step=eps_step,
            decay=None,
            max_iter=100,
            targeted=False,
            num_random_init=0,
            batch_size=512,
            random_eps=False,
            summary_writer=False,
            verbose=True
        ),
        'FastGradient': FastGradientMethod(
            estimator=art_classifier,
            norm=np.inf,
            eps=eps,
            eps_step=eps_step,
            targeted=False,
            num_random_init=0,
            batch_size=512,
            minimal=False,
            summary_writer=False,
        ),
    }

    for attack_name, attack in attacks.items():
        path_curr = f"{path}/Evasion/{attack_name}/eps_{eps_raw:0.4f}"
        pathlib.Path(f"{path_curr}").mkdir(parents=True, exist_ok=True)

        X_adv = attack.generate(np.float32(df.loc[ids_atk, feats].values))
        
        df_adv = df.loc[ids_atk, ['Real']].copy()
        df_adv.loc[ids_atk, feats] = X_adv
        model.produce_probabilities = True
        y_pred_prob = model(torch.from_numpy(np.float32(df_adv.loc[ids_atk, feats].values))).cpu().detach().numpy()
        y_pred = np.argmax(y_pred_prob, 1)
        df_adv["Pred"] = y_pred
        df_adv["Prob Control"] = y_pred_prob[:, 0]
        df_adv["Prob Parkinson"] = y_pred_prob[:, 1]
        df_adv["Eps"] = eps_raw
        df_adv["Data"] = attack_name
        
        for m, drm in dim_red_models.items():
            dim_red_res = drm.transform(df_adv.loc[:, feats].values)
            df_adv.loc[:, dim_red_labels[m][0]] = dim_red_res[:, 0]
            df_adv.loc[:, dim_red_labels[m][1]] = dim_red_res[:, 1]
            
        add_iqr_outs_to_df(df_adv, df.loc[ids_trn_val, :], feats)
        add_pyod_outs_to_df(df_adv, pyod_methods, feats)
            
        df_adv.to_excel(f"{path_curr}/df.xlsx", index_label='sample_id')

        metrics_pred = get_cls_pred_metrics(num_classes=2)
        metrics_prob = get_cls_prob_metrics(num_classes=2)
        df_metrics = pd.DataFrame(index=list(metrics_pred.keys()) + list(metrics_prob.keys()))
        y_real = torch.from_numpy(df_adv.loc[ids_atk, "Real"].values.astype('int32'))
        y_pred_atk = torch.from_numpy(df_adv.loc[ids_atk, "Pred"].values.astype('int32'))
        y_pred_ori = torch.from_numpy(df.loc[ids_atk, "Pred"].values.astype('int32'))
        y_prob_atk = torch.from_numpy(df_adv.loc[ids_atk, ["Prob Control", "Prob Parkinson"]].values)
        y_prob_ori = torch.from_numpy(df.loc[ids_atk, ["Prob Control", "Prob Parkinson"]].values)
        for m in metrics_pred:
            m_val = float(metrics_pred[m][0](y_pred_atk, y_real).numpy())
            metrics_pred[m][0].reset()
            df_metrics.at[m, 'Attack'] = m_val
            m_val = float(metrics_pred[m][0](y_pred_ori, y_real).numpy())
            df_metrics.at[m, 'Origin'] = m_val
            metrics_pred[m][0].reset()
        for m in metrics_prob:
            m_val = 0
            try:
                m_val = float(metrics_prob[m][0](y_prob_atk, y_real).numpy())
            except ValueError:
                pass
            metrics_prob[m][0].reset()
            df_metrics.at[m, 'Attack'] = m_val
            m_val = 0
            try:
                m_val = float(metrics_prob[m][0](y_prob_ori, y_real).numpy())
            except ValueError:
                pass
            metrics_prob[m][0].reset()
            df_metrics.at[m, 'Origin'] = m_val
            
        df_metrics.to_excel(f"{path_curr}/metrics.xlsx", index_label='Metrics')
        
        if attack_name == 'MomentumIterative':
            df_eps.loc[eps_raw, "Origin_Accuracy"] = df_metrics.at['accuracy_weighted', 'Origin']
        df_eps.loc[eps_raw, f"{attack_name}_Accuracy"] = df_metrics.at['accuracy_weighted', 'Attack']
            
df_eps.to_excel(f"{path}/Evasion/df_eps.xlsx", index_label='eps')


In [ ]:
df_eps = pd.read_excel(f"{path}/Evasion/df_eps.xlsx", index_col=0)

atks_trgt = ['MomentumIterative', 'BasicIterative', 'FastGradient']

df_fig = df_eps.loc[:, [f"{x}_Accuracy" for x in atks_trgt]].copy()
df_fig.rename(columns={f"{x}_Accuracy": x for x in atks_trgt}, inplace=True)
df_fig['Eps'] = df_fig.index.values
df_fig = df_fig.melt(id_vars="Eps", var_name='Method', value_name="Accuracy")
fig = plt.figure()
sns.set_theme(style='whitegrid', font_scale=1)
lines = sns.lineplot(
    data=df_fig,
    x='Eps',
    y="Accuracy",
    hue=f"Method",
    style=f"Method",
    palette=colors_atks_eps,
    hue_order=atks_trgt,
    markers=True,
    dashes=False,
)
plt.xscale('log')
lines.set_xlabel(r'$\epsilon$')
x_min = 0.0009
x_max = 1.05
basic = df_eps.at[0.01, f"Origin_Accuracy"]
lines.set_xlim(x_min, x_max)
plt.gca().plot(
    [x_min, x_max],
    [basic, basic],
    color='k',
    linestyle='dashed',
    linewidth=1
)
plt.savefig(f"{path}/Evasion/line_accuracy_vs_eps.png", bbox_inches='tight', dpi=200)
plt.savefig(f"{path}/Evasion/line_accuracy_vs_eps.pdf", bbox_inches='tight')
plt.close(fig)

### Binary Search Steps attacks

In [ ]:
bsss = list(range(1, 11, 1))
df_bss = pd.DataFrame(index=bsss)

for bss in bsss:

    attacks = {
        'ElasticNet': ElasticNet(
            classifier=art_classifier,
            confidence=0.0,
            targeted=False,
            learning_rate=1e-3,
            binary_search_steps=bss,
            max_iter=20,
            beta=1e-3,
            initial_const=1e-4,
            batch_size=1,
            decision_rule="EN",
            verbose=True,
        ),
        'CarliniL2Method': CarliniL2Method(
            classifier=art_classifier,
            confidence=0.0,
            targeted=False,
            learning_rate=0.001,
            binary_search_steps=bss,
            max_iter=20,
            initial_const=1e-4,
            max_halving=5,
            max_doubling=5,
            batch_size=1,
            verbose=True
        ),
        'ZooAttack': ZooAttack(
            classifier=art_classifier,
            confidence=0.0,
            targeted=False,
            learning_rate=0.001,
            max_iter=20,
            binary_search_steps=bss,
            initial_const=1e-4,
            abort_early=True,
            use_resize=False,
            use_importance=True,
            nb_parallel=16,
            batch_size=1,
            variable_h=0.001,
            verbose=True
        ),
    }

    for attack_name, attack in attacks.items():
        path_curr = f"{path}/Evasion/{attack_name}/bss_{bss}"
        pathlib.Path(f"{path_curr}").mkdir(parents=True, exist_ok=True)

        X_adv = attack.generate(np.float32(df.loc[ids_atk, feats].values))
        
        df_adv = df.loc[ids_atk, ['Real']].copy()
        df_adv.loc[ids_atk, feats] = X_adv
        model.produce_probabilities = True
        y_pred_prob = model(torch.from_numpy(np.float32(df_adv.loc[ids_atk, feats].values))).cpu().detach().numpy()
        y_pred = np.argmax(y_pred_prob, 1)
        df_adv["Pred"] = y_pred
        df_adv["Prob Control"] = y_pred_prob[:, 0]
        df_adv["Prob Parkinson"] = y_pred_prob[:, 1]
        df_adv["BSS"] = bss
        df_adv["Data"] = attack_name
        
        for m, drm in dim_red_models.items():
            dim_red_res = drm.transform(df_adv.loc[:, feats].values)
            df_adv.loc[:, dim_red_labels[m][0]] = dim_red_res[:, 0]
            df_adv.loc[:, dim_red_labels[m][1]] = dim_red_res[:, 1]
            
        add_iqr_outs_to_df(df_adv, df.loc[ids_trn_val, :], feats)
        add_pyod_outs_to_df(df_adv, pyod_methods, feats)
            
        df_adv.to_excel(f"{path_curr}/df.xlsx", index_label='sample_id')

        metrics_pred = get_cls_pred_metrics(num_classes=2)
        metrics_prob = get_cls_prob_metrics(num_classes=2)
        df_metrics = pd.DataFrame(index=list(metrics_pred.keys()) + list(metrics_prob.keys()))
        y_real = torch.from_numpy(df_adv.loc[ids_atk, "Real"].values.astype('int32'))
        y_pred_atk = torch.from_numpy(df_adv.loc[ids_atk, "Pred"].values.astype('int32'))
        y_pred_ori = torch.from_numpy(df.loc[ids_atk, "Pred"].values.astype('int32'))
        y_prob_atk = torch.from_numpy(df_adv.loc[ids_atk, ["Prob Control", "Prob Parkinson"]].values)
        y_prob_ori = torch.from_numpy(df.loc[ids_atk, ["Prob Control", "Prob Parkinson"]].values)
        for m in metrics_pred:
            m_val = float(metrics_pred[m][0](y_pred_atk, y_real).numpy())
            metrics_pred[m][0].reset()
            df_metrics.at[m, 'Attack'] = m_val
            m_val = float(metrics_pred[m][0](y_pred_ori, y_real).numpy())
            df_metrics.at[m, 'Origin'] = m_val
            metrics_pred[m][0].reset()
        for m in metrics_prob:
            m_val = 0
            try:
                m_val = float(metrics_prob[m][0](y_prob_atk, y_real).numpy())
            except ValueError:
                pass
            metrics_prob[m][0].reset()
            df_metrics.at[m, 'Attack'] = m_val
            m_val = 0
            try:
                m_val = float(metrics_prob[m][0](y_prob_ori, y_real).numpy())
            except ValueError:
                pass
            metrics_prob[m][0].reset()
            df_metrics.at[m, 'Origin'] = m_val
            
        df_metrics.to_excel(f"{path_curr}/metrics.xlsx", index_label='Metrics')
        
        if attack_name == 'ElasticNet':
            df_bss.loc[bss, "Origin_Accuracy"] = df_metrics.at['accuracy_weighted', 'Origin']
        df_bss.loc[bss, f"{attack_name}_Accuracy"] = df_metrics.at['accuracy_weighted', 'Attack']
            
df_bss.to_excel(f"{path}/Evasion/df_bss.xlsx", index_label='eps')

In [ ]:
df_bss = pd.read_excel(f"{path}/Evasion/df_bss.xlsx", index_col=0)

atks_trgt = ['ElasticNet', 'CarliniL2Method', 'ZooAttack']

df_fig = df_bss.loc[:, [f"{x}_Accuracy" for x in atks_trgt]].copy()
df_fig.rename(columns={f"{x}_Accuracy": x for x in atks_trgt}, inplace=True)
df_fig['BSS'] = df_fig.index.values
df_fig = df_fig.melt(id_vars="BSS", var_name='Method', value_name="Accuracy")
fig = plt.figure()
sns.set_theme(style='whitegrid', font_scale=1)
lines = sns.lineplot(
    data=df_fig,
    x='BSS',
    y="Accuracy",
    hue=f"Method",
    style=f"Method",
    palette=colors_atks_bss,
    hue_order=atks_trgt,
    markers=True,
    dashes=False,
)
lines.set_xlabel('BSS')
basic = pd.read_excel(f"{path}/Evasion/ElasticNet/bss_1/metrics.xlsx", index_col=0).at['accuracy_weighted', 'Origin']
x_min = 0.5
x_max = 10.5
lines.set_xlim(x_min, x_max)
plt.gca().plot(
    [x_min, x_max],
    [basic, basic],
    color='k',
    linestyle='dashed',
    linewidth=1
)
plt.savefig(f"{path}/Evasion/line_accuracy_vs_bss.png", bbox_inches='tight', dpi=200)
plt.savefig(f"{path}/Evasion/line_accuracy_vs_bss.pdf", bbox_inches='tight')
plt.close(fig)

In [ ]:
etas = np.concatenate([np.geomspace(1e-8, 1e-1, 8), np.geomspace(1e-8, 1e-1, 8) * 5])
etas = sorted(etas)

df_etas = pd.DataFrame(index=etas)

for eta in etas:

    attacks = {
        'NewtonFool': NewtonFool(
            classifier=art_classifier,
            max_iter=100,
            eta=eta,
            batch_size=1,
            verbose=True,
        ),
    }

    for attack_name, attack in attacks.items():
        path_curr = f"{path}/Evasion/{attack_name}/eta_{eta:0.2e}"
        pathlib.Path(f"{path_curr}").mkdir(parents=True, exist_ok=True)

        X_adv = attack.generate(np.float32(df.loc[ids_atk, feats].values))
        
        df_adv = df.loc[ids_atk, ['Real']].copy()
        df_adv.loc[ids_atk, feats] = X_adv
        model.produce_probabilities = True
        y_pred_prob = model(torch.from_numpy(np.float32(df_adv.loc[ids_atk, feats].values))).cpu().detach().numpy()
        y_pred = np.argmax(y_pred_prob, 1)
        df_adv["Pred"] = y_pred
        df_adv["Prob Control"] = y_pred_prob[:, 0]
        df_adv["Prob Parkinson"] = y_pred_prob[:, 1]
        df_adv["Eta"] = f"{eta:0.2e}"
        df_adv["Data"] = attack_name
        
        for m, drm in dim_red_models.items():
            dim_red_res = drm.transform(df_adv.loc[:, feats].values)
            df_adv.loc[:, dim_red_labels[m][0]] = dim_red_res[:, 0]
            df_adv.loc[:, dim_red_labels[m][1]] = dim_red_res[:, 1]
            
        add_iqr_outs_to_df(df_adv, df.loc[ids_trn_val, :], feats)
        add_pyod_outs_to_df(df_adv, pyod_methods, feats)
            
        df_adv.to_excel(f"{path_curr}/df.xlsx", index_label='sample_id')

        metrics_pred = get_cls_pred_metrics(num_classes=2)
        metrics_prob = get_cls_prob_metrics(num_classes=2)
        df_metrics = pd.DataFrame(index=list(metrics_pred.keys()) + list(metrics_prob.keys()))
        y_real = torch.from_numpy(df_adv.loc[ids_atk, "Real"].values.astype('int32'))
        y_pred_atk = torch.from_numpy(df_adv.loc[ids_atk, "Pred"].values.astype('int32'))
        y_pred_ori = torch.from_numpy(df.loc[ids_atk, "Pred"].values.astype('int32'))
        y_prob_atk = torch.from_numpy(df_adv.loc[ids_atk, ["Prob Control", "Prob Parkinson"]].values)
        y_prob_ori = torch.from_numpy(df.loc[ids_atk, ["Prob Control", "Prob Parkinson"]].values)
        for m in metrics_pred:
            m_val = float(metrics_pred[m][0](y_pred_atk, y_real).numpy())
            metrics_pred[m][0].reset()
            df_metrics.at[m, 'Attack'] = m_val
            m_val = float(metrics_pred[m][0](y_pred_ori, y_real).numpy())
            df_metrics.at[m, 'Origin'] = m_val
            metrics_pred[m][0].reset()
        for m in metrics_prob:
            m_val = 0
            try:
                m_val = float(metrics_prob[m][0](y_prob_atk, y_real).numpy())
            except ValueError:
                pass
            metrics_prob[m][0].reset()
            df_metrics.at[m, 'Attack'] = m_val
            m_val = 0
            try:
                m_val = float(metrics_prob[m][0](y_prob_ori, y_real).numpy())
            except ValueError:
                pass
            metrics_prob[m][0].reset()
            df_metrics.at[m, 'Origin'] = m_val
            
        df_metrics.to_excel(f"{path_curr}/metrics.xlsx", index_label='Metrics')
        
        if attack_name == 'NewtonFool':
            df_etas.loc[eta, "Origin_Accuracy"] = df_metrics.at['accuracy_weighted', 'Origin']
        df_etas.loc[eta, f"{attack_name}_Accuracy"] = df_metrics.at['accuracy_weighted', 'Attack']
            
df_etas.to_excel(f"{path}/Evasion/df_etas.xlsx", index_label='eta')

In [ ]:
df_etas = pd.read_excel(f"{path}/Evasion/df_etas.xlsx", index_col=0)

atks_trgt = ['NewtonFool']

df_fig = df_etas.loc[:, [f"{x}_Accuracy" for x in atks_trgt]].copy()
df_fig.rename(columns={f"{x}_Accuracy": x for x in atks_trgt}, inplace=True)
df_fig['Eta'] = df_fig.index.values
df_fig = df_fig.melt(id_vars="Eta", var_name='Method', value_name="Accuracy")
fig = plt.figure()
sns.set_theme(style='whitegrid', font_scale=1)
lines = sns.lineplot(
    data=df_fig,
    x='Eta',
    y="Accuracy",
    hue=f"Method",
    style=f"Method",
    palette=colors_atks_eta,
    hue_order=atks_trgt,
    markers=True,
    dashes=False,
)
plt.xscale('log')
lines.set_xlabel(r'$\eta$')
x_min = 8e-9
x_max = 0.6
basic = df_etas.at[0.01, f"Origin_Accuracy"]
lines.set_xlim(x_min, x_max)
plt.gca().plot(
    [x_min, x_max],
    [basic, basic],
    color='k',
    linestyle='dashed',
    linewidth=1
)
plt.savefig(f"{path}/Evasion/line_accuracy_vs_eta.png", bbox_inches='tight', dpi=200)
plt.savefig(f"{path}/Evasion/line_accuracy_vs_eta.pdf", bbox_inches='tight')
plt.close(fig)